In [1]:
// In python use: from pyspark.sql.functions import broadcast, split, lit
import org.apache.spark.sql.functions.{broadcast, split, lit}


val matchesBucketed = spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/matches.csv")
val matchDetailsBucketed =  spark.read.option("header", "true")
                        .option("inferSchema", "true")
                        .csv("/home/iceberg/data/match_details.csv")

Intitializing Scala interpreter ...

Spark Web UI available at http://bfd824c6013a:4044
SparkContext available as 'sc' (version = 3.5.1, master = local[*], app id = local-1737180575566)
SparkSession available as 'spark'


import org.apache.spark.sql.functions.{broadcast, split, lit}
matchesBucketed: org.apache.spark.sql.DataFrame = [match_id: string, mapid: string ... 8 more fields]
matchDetailsBucketed: org.apache.spark.sql.DataFrame = [match_id: string, player_gamertag: string ... 34 more fields]


In [2]:
spark.sql("""DROP TABLE IF EXISTS bootcamp.matches_bucketed""")
val bucketedDDL = """
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id));
 """
 spark.sql(bucketedDDL)

bucketedDDL: String =
"
CREATE TABLE IF NOT EXISTS bootcamp.matches_bucketed (
     match_id STRING,
     is_team_game BOOLEAN,
     playlist_id STRING,
     completion_date TIMESTAMP
 )
 USING iceberg
 PARTITIONED BY (completion_date, bucket(16, match_id));
 "
res0: org.apache.spark.sql.DataFrame = []


In [ ]:
matchesBucketed.select(
     $"match_id", $"is_team_game", $"playlist_id", $"completion_date"
     )
     .write.mode("append")
     .partitionBy("completion_date")
     .bucketBy(16, "match_id").saveAsTable("bootcamp.matches_bucketed")

In [ ]:
 val bucketedDetailsDDL = """
 CREATE TABLE IF NOT EXISTS bootcamp.match_details_bucketed (
     match_id STRING,
     player_gamertag STRING,
     player_total_kills INTEGER,
     player_total_deaths INTEGER
 )
 USING iceberg
 PARTITIONED BY (bucket(16, match_id));
 """
 spark.sql(bucketedDetailsDDL)

In [ ]:
 matchDetailsBucketed.select(
     $"match_id", $"player_gamertag", $"player_total_kills", $"player_total_deaths")
    .write.mode("append")
   .bucketBy(16, "match_id").saveAsTable("bootcamp.match_details_bucketed")

spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")

In [ ]:
matchesBucketed.createOrReplaceTempView("matches")
matchDetailsBucketed.createOrReplaceTempView("match_details")

In [ ]:
spark.sql("""
    SELECT * FROM bootcamp.match_details_bucketed mdb JOIN bootcamp.matches_bucketed md 
    ON mdb.match_id = md.match_id
    AND md.completion_date = DATE('2016-01-01')
        
""").explain()

In [ ]:
spark.sql("""
    SELECT * FROM match_details mdb JOIN matches md ON mdb.match_id = md.match_id
        
""").explain()

In [ ]:
 spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "1000000000000")

In [ ]:
 val broadcastFromThreshold = matches.as("m").join(matchDetails.as("md"), $"m.match_id" === $"md.match_id")
  .select($"m.completion_date", $"md.player_gamertag",  $"md.player_total_kills")
   .take(5)

In [ ]:
 val explicitBroadcast = matches.as("m").join(broadcast(matchDetails).as("md"), $"m.match_id" === $"md.match_id")
   .select($"md.*", split($"completion_date", " ").getItem(0).as("ds"))

In [ ]:
 val bucketedValues = matchDetailsBucketed.as("mdb").join(matchesBucketed.as("mb"), $"mb.match_id" === $"mdb.match_id").explain()
 // .take(5)

In [ ]:
 val values = matchDetailsBucketed.as("m").join(matchesBucketed.as("md"), $"m.match_id" === $"md.match_id").explain()

In [ ]:
 explicitBroadcast.write.mode("overwrite").insertInto("match_details_bucketed")

In [ ]:
 matches.withColumn("ds", split($"completion_date", " ").getItem(0)).write.mode("overwrite").insertInto("matches_bucketed")

In [ ]:
 spark.sql(bucketedSQL)